In [4]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [5]:
a_pos_5289 = "/content/drive/My Drive/Fasta RNA feature vectors/A.thaliana5289_pos/kmer3/A.thaliana5289_pos_kmer_tnc_3.npy"
a_neg_5289 = "/content/drive/My Drive/Fasta RNA feature vectors/A.thaliana5289_neg/kmer3/A.thaliana5289_neg_kmer_tnc.npy"
a_pos_1000 = "/content/drive/My Drive/Fasta RNA feature vectors/A.thaliana1000indep_pos/kmer3/A.thaliana1000indep_pos_kmer_tnc.npy"
a_neg_1000 = "/content/drive/My Drive/Fasta RNA feature vectors/A.thaliana1000indep_neg/kmer_tnc/A.thaliana1000indep_neg_kmer_tnc.npy"

a_x_train_pos = np.load(a_pos_5289) 
a_x_train_neg = np.load(a_neg_5289) 
a_x_test_pos = np.load(a_pos_1000) 
a_x_test_neg = np.load(a_neg_1000) 


b_pos_5289 = "/content/drive/My Drive/Fasta RNA feature vectors/A.thaliana5289_pos/pseknc/A.thaliana5289_pos_pseknc.npy"
b_neg_5289 = "/content/drive/My Drive/Fasta RNA feature vectors/A.thaliana5289_neg/pseknc/A.thaliana5289_neg_pseknc.npy"
b_pos_1000 = "/content/drive/My Drive/Fasta RNA feature vectors/A.thaliana1000indep_pos/pseknc/A.thaliana1000indep_pos_pseknc.npy"
b_neg_1000 = "/content/drive/My Drive/Fasta RNA feature vectors/A.thaliana1000indep_neg/pseknc/A.thaliana1000indep_neg_pseknc.npy"

b_x_train_pos = np.load(b_pos_5289) 
b_x_train_neg = np.load(b_neg_5289) 
b_x_test_pos = np.load(b_pos_1000) 
b_x_test_neg = np.load(b_neg_1000) 

y_train_pos = np.tile(1, 5289) 
y_train_neg = np.tile(0, 5289) 
y_test_pos = np.tile(1, 1000) 
y_test_neg = np.tile(0, 1000) 

In [6]:
print(a_x_test_neg.shape)
print(a_x_train_neg.shape)

print(a_x_test_pos.shape)
print(a_x_train_pos.shape)

print()
print(b_x_test_neg.shape)
print(b_x_train_neg.shape)

print(b_x_test_pos.shape)
print(b_x_train_pos.shape)

(1000, 64)
(5289, 64)
(1000, 64)
(5289, 64)

(1000, 66)
(5289, 66)
(1000, 66)
(5289, 66)


In [9]:
a_x_training = np.concatenate((a_x_train_pos, a_x_train_neg), axis = 0)
a_x_test = np.concatenate((a_x_test_pos, a_x_test_neg), axis = 0)

b_x_training = np.concatenate((b_x_train_pos, b_x_train_neg), axis = 0)
b_x_test = np.concatenate((b_x_test_pos, b_x_test_neg), axis = 0)

y_train = np.concatenate((y_train_pos,y_train_neg))
y_test = np.concatenate((y_test_pos,y_test_neg))

In [10]:
print(a_x_training.shape)
print(a_x_test.shape)

print(b_x_training.shape)
print(b_x_test.shape)

print()
print(y_test.shape)
print(y_train.shape)



(10578, 64)
(2000, 64)
(10578, 66)
(2000, 66)

(2000,)
(10578,)


In [11]:
x_training = np.concatenate((a_x_training, b_x_training), axis = 1)

x_test = np.concatenate((a_x_test, b_x_test), axis = 1)

In [12]:
print(x_training.shape)
print(x_test.shape)

(10578, 130)
(2000, 130)


In [13]:
seed = 40
np.random.seed(seed)

kf = KFold(n_splits=5, random_state=seed, shuffle=True)

In [14]:
test_avg_acc_list = []
j = 1
for c in range(50,502,50):
  RFC = RandomForestClassifier(n_estimators=c,max_depth=None)
  val_accuracy_list = []
  train_accuracy_list = []
  test_acc_list = []
  test_avg = 0
  for train_index, val_index in kf.split(x_training):
    #print("train = ",train_index,"Val =",val_index)
    X_train, X_val = x_training[train_index], x_training[val_index]
    y_train, y_val = y_training[train_index], y_training[val_index]
    
    RFC.fit(X_train, y_train)

    y_val_pred = RFC.predict(X_val)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    val_accuracy_list.append(val_accuracy)

    y_train_pred = RFC.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_accuracy_list.append(train_accuracy)

    y_test_pred = RFC.predict(x_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_acc_list.append(test_accuracy)
    test_avg = test_avg + test_accuracy
  test_avg = test_avg/5
  print(f"Epoch {j}/{10} for C = {c}")
  j=j+1
  for i in range(0, len(train_accuracy_list)):
    print("Train Accuracy : ", train_accuracy_list[i], "\tValidation Accuracy : ", val_accuracy_list[i],"\tTest Accuracy ",test_acc_list[i])
  print("Avg Test Accuracy:",test_avg)

Epoch 1/10 for C = 50
Train Accuracy :  1.0 	Validation Accuracy :  0.9966918714555766 	Test Accuracy  0.738
Train Accuracy :  1.0 	Validation Accuracy :  0.9966918714555766 	Test Accuracy  0.7695
Train Accuracy :  1.0 	Validation Accuracy :  0.998109640831758 	Test Accuracy  0.7715
Train Accuracy :  1.0 	Validation Accuracy :  0.9990543735224586 	Test Accuracy  0.7385
Train Accuracy :  1.0 	Validation Accuracy :  0.9943262411347518 	Test Accuracy  0.7795
Avg Test Accuracy: 0.7594000000000001
Epoch 2/10 for C = 100
Train Accuracy :  1.0 	Validation Accuracy :  0.996219281663516 	Test Accuracy  0.7625
Train Accuracy :  1.0 	Validation Accuracy :  0.997164461247637 	Test Accuracy  0.76
Train Accuracy :  1.0 	Validation Accuracy :  0.998109640831758 	Test Accuracy  0.769
Train Accuracy :  1.0 	Validation Accuracy :  0.9985815602836879 	Test Accuracy  0.7705
Train Accuracy :  1.0 	Validation Accuracy :  0.9943262411347518 	Test Accuracy  0.7465
Avg Test Accuracy: 0.7617
Epoch 3/10 for C = 

In [15]:
test_avg_acc_list = []
j = 1
for c in range(250,300,10):
  RFC = RandomForestClassifier(n_estimators=c,max_depth=None)
  val_accuracy_list = []
  train_accuracy_list = []
  test_acc_list = []
  test_avg = 0
  for train_index, val_index in kf.split(x_training):
    #print("train = ",train_index,"Val =",val_index)
    X_train, X_val = x_training[train_index], x_training[val_index]
    y_train, y_val = y_training[train_index], y_training[val_index]
    
    RFC.fit(X_train, y_train)

    y_val_pred = RFC.predict(X_val)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    val_accuracy_list.append(val_accuracy)

    y_train_pred = RFC.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_accuracy_list.append(train_accuracy)

    y_test_pred = RFC.predict(x_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_acc_list.append(test_accuracy)
    test_avg = test_avg + test_accuracy
  test_avg = test_avg/5
  print(f"Epoch {j}/{10} for C = {c}")
  j=j+1
  for i in range(0, len(train_accuracy_list)):
    print("Train Accuracy : ", train_accuracy_list[i], "\tValidation Accuracy : ", val_accuracy_list[i],"\tTest Accuracy ",test_acc_list[i])
  print("Avg Test Accuracy:",test_avg)

Epoch 1/10 for C = 250
Train Accuracy :  1.0 	Validation Accuracy :  0.9976370510396976 	Test Accuracy  0.7775
Train Accuracy :  1.0 	Validation Accuracy :  0.9985822306238186 	Test Accuracy  0.7685
Train Accuracy :  1.0 	Validation Accuracy :  0.999054820415879 	Test Accuracy  0.7585
Train Accuracy :  1.0 	Validation Accuracy :  0.9995271867612293 	Test Accuracy  0.76
Train Accuracy :  1.0 	Validation Accuracy :  0.9957446808510638 	Test Accuracy  0.7645
Avg Test Accuracy: 0.7657999999999999
Epoch 2/10 for C = 260
Train Accuracy :  1.0 	Validation Accuracy :  0.998109640831758 	Test Accuracy  0.7705
Train Accuracy :  1.0 	Validation Accuracy :  0.9985822306238186 	Test Accuracy  0.772
Train Accuracy :  1.0 	Validation Accuracy :  0.9985822306238186 	Test Accuracy  0.7535
Train Accuracy :  1.0 	Validation Accuracy :  0.9985815602836879 	Test Accuracy  0.7805
Train Accuracy :  1.0 	Validation Accuracy :  0.9957446808510638 	Test Accuracy  0.7665
Avg Test Accuracy: 0.7686
Epoch 3/10 for 

In [16]:
test_avg_acc_list = []
j = 1
for c in range(5,106,10):
  RFC = RandomForestClassifier(n_estimators=270,max_depth=None)
  val_accuracy_list = []
  train_accuracy_list = []
  test_acc_list = []
  test_avg = 0
  for train_index, val_index in kf.split(x_training):
    #print("train = ",train_index,"Val =",val_index)
    X_train, X_val = x_training[train_index], x_training[val_index]
    y_train, y_val = y_training[train_index], y_training[val_index]
    
    RFC.fit(X_train, y_train)

    y_val_pred = RFC.predict(X_val)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    val_accuracy_list.append(val_accuracy)

    y_train_pred = RFC.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_accuracy_list.append(train_accuracy)

    y_test_pred = RFC.predict(x_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_acc_list.append(test_accuracy)
    test_avg = test_avg + test_accuracy
  test_avg = test_avg/5
  print(f"Epoch {j}/{10} for C = {c}")
  j=j+1
  for i in range(0, len(train_accuracy_list)):
    print("Train Accuracy : ", train_accuracy_list[i], "\tValidation Accuracy : ", val_accuracy_list[i],"\tTest Accuracy ",test_acc_list[i])
  print("Avg Test Accuracy:",test_avg)

Epoch 1/10 for C = 5
Train Accuracy :  1.0 	Validation Accuracy :  0.9976370510396976 	Test Accuracy  0.757
Train Accuracy :  1.0 	Validation Accuracy :  0.998109640831758 	Test Accuracy  0.782
Train Accuracy :  1.0 	Validation Accuracy :  0.9995274102079396 	Test Accuracy  0.756
Train Accuracy :  1.0 	Validation Accuracy :  0.9985815602836879 	Test Accuracy  0.767
Train Accuracy :  1.0 	Validation Accuracy :  0.9971631205673759 	Test Accuracy  0.763
Avg Test Accuracy: 0.7649999999999999
Epoch 2/10 for C = 15
Train Accuracy :  1.0 	Validation Accuracy :  0.9966918714555766 	Test Accuracy  0.767
Train Accuracy :  1.0 	Validation Accuracy :  0.9976370510396976 	Test Accuracy  0.761
Train Accuracy :  1.0 	Validation Accuracy :  0.9985822306238186 	Test Accuracy  0.7745
Train Accuracy :  1.0 	Validation Accuracy :  0.9990543735224586 	Test Accuracy  0.774
Train Accuracy :  1.0 	Validation Accuracy :  0.9966903073286052 	Test Accuracy  0.781
Avg Test Accuracy: 0.7715000000000001
Epoch 3/10 